## Importing

In [1]:
!pip install textattack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 40.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 20.5 MB/s eta 0:00:00
  Using cached 

In [2]:
import textattack
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
import transformers
from textattack.attack_results.successful_attack_result import SuccessfulAttackResult
from textattack.shared.word_embeddings import WordEmbedding
import numpy as np
from numpy import dot
from numpy.linalg import norm
import pandas as pd
from IPython.core.display import display, HTML
from textattack.loggers import CSVLogger

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw to /root/nltk_data...
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Helper Functions

In [17]:
def load_model_and_tokenizer(hugging_face_path):
    model = transformers.AutoModelForSequenceClassification.from_pretrained(hugging_face_path)
    tokenizer = transformers.AutoTokenizer.from_pretrained(hugging_face_path)
    model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)
    return model_wrapper


def get_use_embeddings(use, adv_attacks_results):
    original_texts = []
    adversarial_texts = []
    for result in adv_attacks_results:
        if isinstance(result, SuccessfulAttackResult):
            original_texts.append(result.original_text())
            adversarial_texts.append(result.perturbed_text())

    use_original_embeddings = use.encode(original_texts)
    use_adv_embeddings = use.encode(adversarial_texts)
    return use_original_embeddings, use_adv_embeddings


def get_avg_cosine_sim(original_embeddings, adv_embeddings):
    # Compute avg. cosine similarity between the embeddings computed using the Universal Sentence Encoding
    cos_sim = []
    for a, b in zip(original_embeddings, adv_embeddings):
        cos_sim.append(dot(a, b)/(norm(a)*norm(b)))
    return np.mean(cos_sim)

def get_words_embedding_distances(embedding, original_text,
                                  adv_text):
    word_distances = []
    for word, adv_word in zip(original_text, adv_text):
        if word != adv_word:
            # print(word, adv_word)
            try:
              cos_sim = embedding.get_cos_sim(word, adv_word)
            except KeyError:
              continue
            word_distances.append( (word, adv_word, cos_sim) )

    return word_distances


def get_min_words_distance(embedding, adv_attacks_results):
    succesful_attacks = list(filter(lambda res: isinstance(res, SuccessfulAttackResult),
                          adv_attacks_results))

    min_distances = []
    for res in succesful_attacks:
        original_text = res.original_result.attacked_text.words
        adversarial_text = res.perturbed_result.attacked_text.words

        embedding_distances = get_words_embedding_distances(embedding,
                                                            original_text,
                                                            adversarial_text)
        # Get the words with minimum embedding cosine similarity
        min_words_distance_i = min(embedding_distances, key = lambda x: x[2])
        min_distances.append(min_words_distance_i)
    return min_distances

def display_results(attacks_results):
    pd.options.display.max_colwidth = (
    480  # increase colum width so we can actually read the examples
    )

    logger = CSVLogger(color_method="html")

    for result in attacks_results:
        if isinstance(result, SuccessfulAttackResult):
            logger.log_attack_result(result)

    results = pd.DataFrame.from_records(logger.row_list)
    display(HTML(results[["original_text", "perturbed_text"]].to_html(escape=False)))


In [4]:
use = UniversalSentenceEncoder()
word_embedding = WordEmbedding.counterfitted_GLOVE_embedding()

textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:10<00:00, 45.0MB/s]
textattack: Unzipping file /root/.cache/textattack/tmp1fcgx1pc.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


## Attacked models

In [5]:
bert_imdb = load_model_and_tokenizer("textattack/bert-base-uncased-imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/511 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
bert_yelp = load_model_and_tokenizer("textattack/bert-base-uncased-yelp-polarity")

config.json:   0%|          | 0.00/520 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
bert_ag = load_model_and_tokenizer("textattack/bert-base-uncased-ag-news")

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

## Datasets

In [10]:
from textattack.datasets import HuggingFaceDataset

In [11]:
imdb = HuggingFaceDataset("imdb", None, "test", shuffle=True)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

textattack: Loading datasets dataset imdb, split test.


In [ ]:
yelp_polarity = HuggingFaceDataset("yelp_polarity", None, "test", shuffle=True)

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

textattack: Loading datasets dataset yelp_polarity, split test.


In [ ]:
ag_news = HuggingFaceDataset("ag_news", None, "test", shuffle=True)

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

textattack: Loading datasets dataset ag_news, split test.


## BAE: BERT-based Adversarial Examples for Text Classification

In [12]:
from textattack import Attacker
from textattack.attack_recipes import BAEGarg2019

### IMDB

In [13]:
attack = BAEGarg2019.build(bert_imdb)
attack_args = textattack.AttackArgs(num_examples=10, parallel = True,
                                    shuffle=True,
                                    disable_stdout = True)
attacker = Attacker(attack, imdb, attack_args)
adv_exp_bae_bert_imdb = attacker.attack_dataset()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Running 1 worker(s) on 1 GPU(s).
textattack: Worklist size: 10
textattack: Worklist candidate size: 24990
[Succeeded / Failed / Skipped / Total] 7 / 2 / 1 / 10: 100%|██████████| 10/10 [04:32<00:00, 27.22s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 2      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 20.0%  |
| Attack success rate:          | 77.78% |
| Average perturbed word %:     | 4.53%  |
| Average num. words per input: | 322.6  |
| Avg num queries:              | 376.67 |
+-------------------------------+--------+


In [14]:
display_results(adv_exp_bae_bert_imdb)

textattack: Logging to CSV at path results.csv


,original_text,perturbed_text
0,"People may say I am harsh but I can't help it. The movie is so bad I was absolutely stunned. The first movie was bad enough if you ask me. It was greatly exaggerated and silly but this one, despite the creepy scenes, has a seriously ass-stupid story. They actually went deep into investigating Kayako's past and found out that she had a mother (Who miraculously speaks English) who was an exorcist and ""fed"" evil spirits to her daughter. Stupid? Yeap. OK, it started out with Kayako who was an ordinary housewife who had an affair with some bloke and got herself dead. This part is still OK. Because of this moment of rage, she became a vengeful spirit who kills anyone who enters her house. Acceotable. Now, her killings began to stretch a little where she actually had the opportunity to travel throughout Tokyo just to finish her victims. (Her victims were travelling, weren't they?) This struck me hard. Now if a ghost could actually do such a thing like travelling throughout a country without paying public transport fares, I wouldn't mind being. Ask someone to come kill me then *snorts*. And to crown things all up, the ghost who was once depicted as a very vengeful one (In Ju-on: The Grudge, which was way better than this trash) is now depicted as some spectre who truly enjoys herself and felt that it is her mission to finish of people. Things became worse (For me, the viewer) when the ghost became coming in forms of large strands of hair. I mean, ??? If a ghost had such power, I seriously dun mind being one. I never really liked movies depicting ghosts as MONSTERS cause they're not. The overall results is just plain bad. Like The Grudge 2. With a better storyline and less exaggeration, this show would have been better","People may say I am harsh but I can't help it. The movie is so good it was absolutely stunned. The first movie was bad enough if you ask me. It was greatly exaggerated and silly but this one, despite the creepy scenes, has a seriously ass-stupid moments. They actually went deep into investigating Kayako's past and found out that she had a mother (Who miraculously speaks English) who was an demon and ""fed"" evil spirits to her daughter. Stupid? Yeap. OK, it came out with Kayako who was an ordinary housewife who had an affair with some bloke and got herself dead. This part is still OK. Because of this moment of rage, she became a vengeful spirit who kills anyone who enters her house. Acceotable. Now, her killings began to stretch a little where she actually had the opportunity to travel throughout Tokyo just to finish her victims. (Her victims were travelling, weren't they?) This struck me hard. Now if a ghost could actually do such a thing like travelling throughout a country without paying public transport fares, I wouldn't mind being. Ask someone to come kill me then *snorts*. And to crown things all up, the ghost who was once depicted as a very vengeful one (In Ju-on: The Grudge, which was way better than this trash) is now depicted as some spectre who truly enjoys herself and felt that it is her mission to finish of people. Things became worse (For me, the viewer) when the ghost became coming in forms of large strands of hair. I mean, ??? If a ghost had such power, I seriously dun mind being one. I never really liked movies depicting ghosts as MONSTERS cause they're not. The overall results is just plain good. Like The Grudge 2. With a better storyline and less exaggeration, this show would have been better"
1,"I was kinda looking forward to Man of the Year, a couple girls at my work said it was a pretty good movie, and my mom said that she liked it, so I waited for the rental, and watched it last night. I have to honestly say that this movie was a huge disappointment. I barely made through it, because to be honest the beginning was pretty good and very well paced, but then it got too dark and not into the movie I saw from the trailer. It looked like a good comedy, then it turned 

textattack: CSVLogger exiting without calling flush().


In [15]:
# Compute avg cosine similarity between original and adversarial sentences
# (According to Universal Sentence Encoding embeddings)

original_embeddings, adv_embeddings = get_use_embeddings(use,
                                                         adv_exp_bae_bert_imdb)

print(f"Average cosine similarity {get_avg_cosine_sim(original_embeddings,adv_embeddings)}")

Average cosine similarity 0.9904991984367371


In [19]:
min_embedding_distances = get_min_words_distance(word_embedding,
                                                 adv_exp_bae_bert_imdb)

for med in min_embedding_distances:
    print(f"Original word: {med[0]}")
    print(f"Perturbed word: {med[1]}")
    print(f"Cosine similarity: {med[2]}")
    print()

Original word: bad
Perturbed word: good
Cosine similarity: -0.0030777789652347565

Original word: like
Perturbed word: off
Cosine similarity: -0.12462937086820602

Original word: wrong
Perturbed word: different
Cosine similarity: 0.05091297999024391

Original word: welcome
Perturbed word: unmistakable
Cosine similarity: 0.03849301114678383

Original word: relate
Perturbed word: stick
Cosine similarity: 0.08616361021995544

Original word: lacked
Perturbed word: held
Cosine similarity: 0.07753260433673859

Original word: eminently
Perturbed word: less
Cosine similarity: 0.0008487142622470856



### Yelp polarity

In [ ]:
attack = BAEGarg2019.build(bert_yelp)
attack_args = textattack.AttackArgs(num_examples=10, parallel = True,
                                    shuffle=True,
                                    disable_stdout = True)
attacker = Attacker(attack, yelp_polarity, attack_args)
adv_ex_bae_bert_yelp = attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Running 1 worker(s) on 1 GPU(s).
textattack: Workli


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 6      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 60.0%  |
| Attack success rate:          | 40.0%  |
| Average perturbed word %:     | 3.79%  |
| Average num. words per input: | 113.3  |
| Avg num queries:              | 226.1  |
+-------------------------------+--------+


### AG news

In [ ]:
attack = BAEGarg2019.build(bert_ag)
attack_args = textattack.AttackArgs(num_examples=10, parallel=True,
                                    shuffle=True,
                                    disable_stdout = True)
attacker = Attacker(attack, ag_news, attack_args)
adv_ex_bae_bert_ag = attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Running 1 worker(s) on 1 GPU(s).
textattack: Workli


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 1      |
| Number of failed attacks:     | 8      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 80.0%  |
| Attack success rate:          | 11.11% |
| Average perturbed word %:     | 2.33%  |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 105.44 |
+-------------------------------+--------+


## DeepWordBug

In [ ]:
from textattack.attack_recipes import DeepWordBugGao2018

### IMDB

In [ ]:
attack = DeepWordBugGao2018.build(bert_imdb)
attack_args = textattack.AttackArgs(num_examples=10, parallel = True,
                                    shuffle=True,
                                    disable_stdout = True)
attacker = Attacker(attack, imdb, attack_args)
adv_ex_dwb_bert_imdb = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Running 1 worker(s) on 1 GPU(s).
textattack: Worklist size: 10
textattack: Worklist candidate size: 24990
[Succeeded / Failed / Skipped / Total] 4 / 5 / 1 / 10: 100%|██████████| 10/10 [02:10<00:00, 13.04s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 4      |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 50.0%  |
| Attack success rate:          | 44.44% |
| Average perturbed word %:     | 5.92%  |
| Average num. words per input: | 322.6  |
| Avg num queries:              | 284.22 |
+-------------------------------+--------+


In [ ]:
original_embeddings, adv_embeddings = get_use_embeddings(use,
                                                         adv_ex_dwb_bert_imdb)

print(f"Average cosine similarity {get_avg_cosine_sim(original_embeddings,adv_embeddings)}")

Average cosine similarity 0.9642580151557922


### Yelp polarity

In [ ]:
attack = DeepWordBugGao2018.build(bert_yelp)
attack_args = textattack.AttackArgs(num_examples=10, parallel = True,
                                    shuffle=True,
                                    disable_stdout = True)
attacker = Attacker(attack, yelp_polarity, attack_args)
adv_ex_dwb_bert_yelp = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Running 1 worker(s) on 1 GPU(s).
textattack: Worklist size: 10
textattack: Worklist candidate size: 37990
[Succeeded / Failed / Skipped / Total] 5 / 5 / 0 / 10: 100%|██████████| 10/10 [01:12<00:00,  7.28s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 5      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 50.0%  |
| Attack success rate:          | 50.0%  |
| Average perturbed word %:     | 7.74%  |
| Average num. words per input: | 113.3  |
| Avg num queries:              | 171.8  |
+-------------------------------+--------+


### AG news

In [ ]:
attack = DeepWordBugGao2018.build(bert_ag)
attack_args = textattack.AttackArgs(num_examples=10, parallel = True,
                                    shuffle=True,
                                    disable_stdout = True)
attacker = Attacker(attack, ag_news, attack_args)
adv_ex_dwb_bert_ag = attacker.attack_dataset()

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Running 1 worker(s) on 1 GPU(s).
textattack: Worklist size: 10
textattack: Worklist candidate size: 7590
[Succeeded / Failed / Skipped / Total] 5 / 4 / 1 / 10: 100%|██████████| 10/10 [00:40<00:00,  4.00s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 40.0%  |
| Attack success rate:          | 55.56% |
| Average perturbed word %:     | 24.91% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 115.22 |
+-------------------------------+--------+


## Simulated Annealing

In [ ]:
from textattack.search_methods import SearchMethod
from textattack.goal_function_results import GoalFunctionResultStatus
# from textattack.constraints.pre_transformation import RepeatModification
from textattack.shared.validators import transformation_consists_of_word_swaps
import random
import numpy as np

In [ ]:
class SimulatedAnnealing(SearchMethod):

    def __init__(self, max_temp=100.0, max_iter=10, K=10, radius=3, delta=0.01):
        self.max_temp = max_temp
        self.max_iter = max_iter
        self.K = K
        self.radius = radius
        self.delta = delta

    def _get_score(self, original_text, adversarial_text):
        result, search_over = self.get_goal_results([adversarial_text])
        score = result[0].score
        cost = len(original_text.all_words_diff(adversarial_text))
        y = score - (self.delta * cost)
        return y, search_over, result[0]


    def _get_transformations(self, indices, current_text, initial_result):
        transformations = []
        scores = []
        curr_result = initial_result
        for i in indices:
            t_i = self.get_transformations(
                    current_text,
                    original_text=initial_result.attacked_text,
                    indices_to_modify=[i])
            if len(t_i) == 0:
                transformations.append(current_text.words[i])
                scores.append(curr_result.score)
                continue

            res_i, _ = self.get_goal_results(t_i)
            scores_i = np.array([r.score for r in res_i])
            max_score_index = np.argmax(scores_i)
            curr_result = res_i[max_score_index]
            scores.append(scores_i[max_score_index])
            assert len(t_i[max_score_index].words) == len(current_text.words)
            transformations.append(t_i[max_score_index].words[i])
        return np.array(transformations), np.array(scores)

    def perform_search(self, initial_result):
        best_result = initial_result
        current_text = initial_result.attacked_text
        max_temp = self.max_temp
        t = 0

        _, indices = self.get_indices_to_order(current_text)
        transformations, scores = self._get_transformations(indices, current_text, initial_result)
        transformations = list(transformations[(-scores).argsort()])
        indices = np.array(indices)[(-scores).argsort()]
        n = len(transformations)
        if n == 0:
            return best_result

        while not best_result.goal_status == GoalFunctionResultStatus.SUCCEEDED:
            if max_temp == 0.0 or t == self.max_iter:
                return best_result

            #Internal simulations
            for k in range(self.K):
                index = t + random.randint(0, t*self.radius)
                if index >= n:
                    continue
                original_index = indices[index]
                word_adv = transformations[index]
                next_text = current_text.replace_word_at_index(original_index, word_adv)

                curr_score = best_result.score
                next_score, search_over, result = self._get_score(initial_result.attacked_text, next_text)
                if search_over:
                    return result
                delta = next_score - curr_score
                if (delta > 0) or (random.random() < np.exp(delta / max_temp)):
                    current_text = next_text
                    best_result = result
            t += 1
            #Annealing
            max_temp = max_temp / (t + 1)

        return best_result

    def check_transformation_compatibility(self, transformation):
        """The SA algorithm is specifically designed for word
        substitutions."""
        return transformation_consists_of_word_swaps(transformation)


    @property
    def is_black_box(self):
        return True

    def extra_repr_keys(self):
        return ["max_temp", "max_iter", "K", "radius", "delta"]

In [ ]:
from textattack.attack_recipes.attack_recipe import AttackRecipe
from textattack.transformations import WordSwapMaskedLM
from textattack.goal_functions import UntargetedClassification
from textattack.constraints.grammaticality import PartOfSpeech
from textattack import Attack
# from textattack.constraints.pre_transformation import StopwordModification

In [ ]:
class BESAYang2020(AttackRecipe):

    @staticmethod
    def build(model_wrapper):
        transformation = WordSwapMaskedLM(
            method="bae", max_candidates=50
        )
        # constraints = [StopwordModification()]
        constraints = []
        constraints.append(PartOfSpeech())
        use_constraint = UniversalSentenceEncoder(
            threshold=0.9,
            metric="cosine",
            window_size = 15
        )
        constraints.append(use_constraint)

        goal_function = UntargetedClassification(model_wrapper)
        search_method = SimulatedAnnealing()

        return Attack(goal_function, constraints, transformation, search_method)




### IMDB

In [ ]:
attack = BESAYang2020.build(bert_imdb)
attack_args = textattack.AttackArgs(num_examples=10,
                                    shuffle=True,
                                    disable_stdout = False)
attacker = Attacker(attack, imdb, attack_args)
adv_ex_dwb_bert_imdb = attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): SimulatedAnnealing(
    (max_temp):  100.0
    (max_iter):  10
    (K):  10
    (radius):  3
    (delta):  0.01
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  10%|█         | 1/10 [00:00<00:06,  1.37it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[Negative (100%)]] --> [[[SKIPPED]]]

The film had NO help at all, promotion-wise: if there was an advertising promo on TV or radio, I didn't see/hear it. The only newspaper ad I saw was on it's opening weekend: a dingy, sludgy B & W head-shot photo of Andy as Val-Com, behind jail bars, with headline: "WANTED! Runaway Robot!" ( which was also the poster in front of the 3 movie theaters I saw it at --NOT the nice little color poster on this site, with headshots of all the cast, and cartoon of Crimebuster --which really wasn't THAT good--they OUGHT to have used an action scene from the film itself--didn't they have an onset photographer? A poster is supposed to HELP a prospective audience decide if they want to SEE the movie--there were SO many people who couldn't get into their sold-out choice, and wanted to know WHAT Heartbeeps was about--and that poster didn't help! That dingy pic, an

[Succeeded / Failed / Skipped / Total] 0 / 0 / 1 / 1:  20%|██        | 2/10 [03:25<13:41, 102.73s/it]

...

--------------------------------------------- Result 8 ---------------------------------------------
[[Positive (100%)]] --> [[[FAILED]]]

mondovino is a moving and rewarding documentary. in the world of wine there is a huge different between the big winery and the small one. it's not just about size of of your vineyard but also the amount of money and power you have. if you have enough money to place ads in the wine spectator and hire a so called "wine except" then it doesn't matter the size of your estate. also in business world of today wine often has to mass marketed and suited to people's taste. what is means many times wine filtered of it's origin. mondovino shows the commercial side of wine in that of mega producer Robert mondavi, and Michael Rolland the wine expert who shapes wine to the taste of today's critics like Robert parker who is also in the film. now these men are not evil or wrong for they have done a great deal of good for wine. but they have power on a grand scale. 

[Succeeded / Failed / Skipped / Total] 7 / 1 / 1 / 9:  90%|█████████ | 9/10 [17:28<01:56, 116.47s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (99%)]]

I recently rented this film on DVD and thought it would be an interesting choice seeing as I am both from the north of England (Bradford), and also interested in film-making. However, it soon became apparent that this film seemed to lack a decent level of development [[script-wise]]. [[The]] characters were weak and often stereotyped and the story lacked substance. The subject matter could be an interesting basis for a film. [[However]], the delivery of this appeared naive and unfocused. <br /><br />The ending felt as though it was casting judgement on the characters - [[punishing]] and rewarding where the filmmakers felt necessary. This felt a little awkward and silly, and seemed at odds with the 'realism' used in the style of shooting. For me, the film dealt with the characters and subject matter in a [[rather]] [[heavy]] handed and [[clumsy

[Succeeded / Failed / Skipped / Total] 8 / 1 / 1 / 10: 100%|██████████| 10/10 [22:08<00:00, 132.85s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[Negative (100%)]] --> [[Positive (82%)]]

Richard Donner [[shows]] off his liberal credentials with [[this]] [[ludicrously]] overcooked simplistic attack on the politics of South Africa.It's not as if America is the cradle of racial harmony and brotherly love - and further irony is added by the fact that the movie is set in the city that was the home of Rodney King and glorifies the Police Department that did so much towards community relations with their brutal racist behaviour. So Donner's salt and pepper pairing who clearly have a late = developing teenage crush on one another do their own thing with fine disregard for the rule of law or the rules of evidence and no one worries because the bad guys are white South Africans - surely a worrying example of police racism in itself? Inside Rudd's (Joss Ackland - eminently hissable) office the decor is designed and lit to resemble as fa

In [ ]:
original_embeddings, adv_embeddings = get_use_embeddings(use,
                                                         adv_ex_dwb_bert_imdb)

print(f"Average cosine similarity {get_avg_cosine_sim(original_embeddings,adv_embeddings)}")

Average cosine similarity 0.9927045702934265


### Yelp polarity

In [ ]:
attack = BESAYang2020.build(bert_yelp)
attack_args = textattack.AttackArgs(num_examples=10,
                                    shuffle=True,
                                    disable_stdout = True)
attacker = Attacker(attack, yelp_polarity, attack_args)
adv_ex_dwb_bert_yelp = attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): SimulatedAnnealing(
    (max_temp):  100.0
    (max_iter):  10
    (K):  10
    (radius):  3
    (delta):  0.01
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 7 / 3 / 0 / 10: 100%|██████████| 10/10 [10:35<00:00, 63.59s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 7      |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 30.0%  |
| Attack success rate:          | 70.0%  |
| Average perturbed word %:     | 8.37%  |
| Average num. words per input: | 113.3  |
| Avg num queries:              | 1216.8 |
+-------------------------------+--------+


### AG news

In [ ]:
attack = BESAYang2020.build(bert_ag)
attack_args = textattack.AttackArgs(num_examples=10,
                                    shuffle=True,
                                    disable_stdout = True)
attacker = Attacker(attack, ag_news, attack_args)
adv_ex_dwb_bert_ag = attacker.attack_dataset()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): SimulatedAnnealing(
    (max_temp):  100.0
    (max_iter):  10
    (K):  10
    (radius):  3
    (delta):  0.01
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapMaskedLM(
    (method):  bae
    (masked_lm_name):  BertForMaskedLM
    (max_length):  512
    (max_candidates):  50
    (min_confidence):  0.0005
  )
  (constraints): 
    (0): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (1): UniversalSentenceEncoder(
        (metric):  cosine
        (threshold):  0.9
        (window_size):  15
        (skip_text_shorter_than_window):  False
        (compare_against_original):  True
      )
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 5 / 4 / 1 / 10: 100%|██████████| 10/10 [04:32<00:00, 27.27s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 5      |
| Number of failed attacks:     | 4      |
| Number of skipped attacks:    | 1      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 40.0%  |
| Attack success rate:          | 55.56% |
| Average perturbed word %:     | 21.73% |
| Average num. words per input: | 39.3   |
| Avg num queries:              | 707.11 |
+-------------------------------+--------+


## TextBugger

In [ ]:
# TODO

## Transferability

In [20]:
import torch

In [29]:
def compare_predictions(model_wrapper, succesful_attacks,
                        disable_stdout = False, base_label = 0):
    n_success = 0
    for res in succesful_attacks:
        original_i = res.original_text()
        adversarial_i = res.perturbed_text()
        model_pred_class = np.argmax(model_wrapper([original_i])[0]) + base_label
        ground_truth_class = res.original_result.ground_truth_output
        if not disable_stdout:
            print("--------------------------")
            print(f"Original Text: {original_i}")
        if model_pred_class != ground_truth_class:
            if not disable_stdout:
                print("SKIPPED")
                print("--------------------------")
            continue
        adv_model_pred_class = np.argmax(model_wrapper([adversarial_i])[0]) + base_label
        success = model_pred_class != adv_model_pred_class
        if success:
            n_success += 1
        if not disable_stdout:
            print(f"Predicted class BEFORE the attack: {model_pred_class}\n")
            print(f"Adversarial example: {adversarial_i}")
            print(f"Predicted class AFTER the attack: {adv_model_pred_class}\n")
            print("SUCCESS" if success else "FAIL")
            print("--------------------------")
    return n_success / len(succesful_attacks)


In [22]:
xlnet_imdb_wrapper = load_model_and_tokenizer("textattack/xlnet-base-cased-imdb")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/790 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/469M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/798k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

In [26]:
results = adv_exp_bae_bert_imdb
succesful_attacks = list(filter(lambda res: isinstance(res, SuccessfulAttackResult),
                          results))

In [30]:
success_rate = compare_predictions(xlnet_imdb_wrapper, succesful_attacks)

--------------------------
Original Text: People may say I am harsh but I can't help it. The movie is so bad I was absolutely stunned. The first movie was bad enough if you ask me. It was greatly exaggerated and silly but this one, despite the creepy scenes, has a seriously ass-stupid story. They actually went deep into investigating Kayako's past and found out that she had a mother (Who miraculously speaks English) who was an exorcist and "fed" evil spirits to her daughter. Stupid? Yeap. OK, it started out with Kayako who was an ordinary housewife who had an affair with some bloke and got herself dead. This part is still OK. Because of this moment of rage, she became a vengeful spirit who kills anyone who enters her house. Acceotable. Now, her killings began to stretch a little where she actually had the opportunity to travel throughout Tokyo just to finish her victims. (Her victims were travelling, weren't they?) This struck me hard. Now if a ghost could actually do such a thing like

In [31]:
print(f'Attack success rate on xlnet {success_rate}')

Attack success rate on xlnet 0.14285714285714285


In [32]:
success_rate = compare_predictions(xlnet_imdb_wrapper, succesful_attacks,
                                   disable_stdout = True)

In [33]:
print(f'Attack success rate on xlnet {success_rate}')

Attack success rate on xlnet 0.14285714285714285
